In [1]:
import findspark
findspark.init()

In [2]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder  \
        .master('local[*]') \
        .appName('Group_by') \
        .getOrCreate()

22/02/24 10:46:09 WARN Utils: Your hostname, sebastian2296 resolves to a loopback address: 127.0.1.1; using 192.168.20.23 instead (on interface wlp2s0)
22/02/24 10:46:09 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/02/24 10:46:10 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df_green = spark.read.parquet('../tripdata/green/repartitioned/*/*')

In [4]:
df_green = df_green \
.withColumnRenamed('lpep_pickup_datetime', 'pickup_datetime') \
.withColumnRenamed('lpep_dropoff_datetime', 'dropoff_datetime')

In [5]:
df_green.createOrReplaceTempView('green')

In [11]:
df_result = spark.sql(''' SELECT

    date_trunc('hour', pickup_datetime) AS hour,  
    PULocationID AS zone,
    

    SUM(total_amount) AS amount,
    COUNT(1) as number_records
    FROM
    green
    WHERE pickup_datetime >= '2020-01-01 00:00:00'
    GROUP BY
    1, 2
    ORDER BY
    1, 2''')

In [12]:
df_result.show()

+-------------------+----+------------------+--------------+
|               hour|zone|            amount|number_records|
+-------------------+----+------------------+--------------+
|2020-01-01 00:00:00|   7| 368.7800000000001|            18|
|2020-01-01 00:00:00|  16|              20.3|             1|
|2020-01-01 00:00:00|  17|              58.3|             3|
|2020-01-01 00:00:00|  25|               7.2|             1|
|2020-01-01 00:00:00|  31|             23.34|             1|
|2020-01-01 00:00:00|  35|109.27000000000001|             2|
|2020-01-01 00:00:00|  36|            144.93|             8|
|2020-01-01 00:00:00|  37|            160.89|             5|
|2020-01-01 00:00:00|  39|             66.22|             2|
|2020-01-01 00:00:00|  40|              13.8|             1|
|2020-01-01 00:00:00|  41|             50.45|             4|
|2020-01-01 00:00:00|  42|             94.17|            10|
|2020-01-01 00:00:00|  56|              8.84|             1|
|2020-01-01 00:00:00|  6

In [13]:
!cd data_all && mkdir group_by

In [14]:
df_result.write.parquet('data_all/group_by/', mode = 'overwrite')

22/02/24 10:56:12 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
